In [ ]:
import os
os.chdir("../src/")

# POS + ROBERTA TRAIN

In [ ]:

from bert_tfidf import *
import pandas as pd
train_df = pd.read_csv('../data/liwc_pos_dep_tr.csv')
test_df = pd.read_csv('../data/liwc_pos_dep_eval.csv')
reddit_df = pd.read_csv('../data/liwc_pos_dep_reddit.csv')

labels2id = {l: i for i, l in enumerate(train_df['alg'].unique())}

concat_model, vectorizer, train_dataloader, val_dataloader, test_dataloader, reddit_dataloader = data_batcher(train_df, test_df, reddit_df, labels2id, tfidf_fields=["pos_sequence"], count_fields=["pos_sequence"], batch_size=24, model_name="roberta-base")

final_model, best_model = modelling(concat_model, train_dataloader, val_dataloader, "roberta_base_pos", vectorizer, 5, parallel_gpu=True)

evaluate_test(final_model, reddit_dataloader, labels2id, reddit_df["alg"])

evaluate_test(final_model, test_dataloader, labels2id, test_df["alg"])

# POS + DEP + ROBERTA

In [ ]:
from bert_tfidf import *
import pandas as pd

train_df = pd.read_csv('../data/liwc_pos_dep_tr.csv')
test_df = pd.read_csv('../data/liwc_pos_dep_eval.csv')
reddit_df = pd.read_csv('../data/liwc_pos_dep_reddit.csv')

labels2id = {l: i for i, l in enumerate(train_df['alg'].unique())}

concat_model, vectorizer, train_dataloader, val_dataloader, test_dataloader, reddit_dataloader =data_batcher(train_df, test_df, reddit_df, labels2id, tfidf_fields=["pos_sequence", "dep"], count_fields=["pos_sequence", "dep"], batch_size=24, model_name="roberta-base")

final_model, best_model = modelling(concat_model, train_dataloader, val_dataloader, "roberta_base_pos_dep", vectorizer, 5, parallel_gpu=True)

evaluate_test(final_model, test_dataloader, labels2id, test_df["alg"])
evaluate_test(final_model, reddit_dataloader, labels2id, reddit_df["alg"])

# POS + DEP + Linguistic + Roberta

In [ ]:
from bert_tfidf import *
import pandas as pd
train_df = pd.read_csv('../data/liwc_pos_dep_tr.csv')
test_df = pd.read_csv('../data/liwc_pos_dep_eval.csv')
reddit_df = pd.read_csv('../data/liwc_pos_dep_reddit.csv')

labels2id = {l: i for i, l in enumerate(train_df['alg'].unique())}
concat_model, vectorizer, train_dataloader, val_dataloader, test_dataloader, reddit_dataloader =data_batcher(train_df, test_df, reddit_df, labels2id, tfidf_fields=["pos_sequence", "dep"], count_fields=["pos_sequence", "dep"], numerical_fields=["semantic_coherence", "Analytic", "WPS", "article", "Period"], batch_size=24, model_name="roberta-base")

final_model, best_model = modelling(concat_model, train_dataloader, val_dataloader, "roberta_base_pos_dep_liwc", vectorizer, 5, parallel_gpu=True)
evaluate_test(final_model, test_dataloader, labels2id, test_df["alg"])
evaluate_test(final_model, reddit_dataloader, labels2id, reddit_df["alg"])


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

def return_cm(y_true, y_pred, reddit=False):
  if reddit:
      classes = ["human", "gpt3", "instructgpt"]
  else:
      classes = ["ctrl", "fair", "gpt", "gpt2", "gpt3", "grover", "human", "instructgpt", "pplm", "xlm", "xlnet"]

  cm = confusion_matrix(y_true, y_pred, labels=classes)
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title('Confusion Matrix')
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)
  thresh = cm.max() / 2.
  for i, j in np.ndindex(cm.shape):
      plt.text(j, i, format(cm[i, j], 'd'),
              horizontalalignment="center",
              color="white" if cm[i, j] > thresh else "black")
  plt.xlabel('Predicted Label')
  plt.ylabel('True Label')
  return plt
